MIKE MINNEBACH - 500903092 - 

In [338]:
import re #to write regex
import pandas as pd #for the dataframe
import datetime #for date formatting

Convert a MT103 SWIFT message 
(https://www2.swift.com/knowledgecentre/products/Standards%20MT) into the 
following data structure. Write a code in your chosen language to convert the message to 
the following data structure. The notEmpty fields are mandatory. (25 points)
Converting each of the mandatory fields correctly carries 1 point and converting each of 
the non-mandatory fields contains 0.5 points (15+8.5+1.5). 1.5 points are given for coding 
style.

In [339]:
file = open("MT103.txt", 'r') #read the file

In [340]:
#initiate lookup variables
transaction_re = re.compile(r'transaction_|instrument_|originator_|beneficiary_|ingoing_|outgoing_*') #regex to identify the items which are needed in the dataframe


In [341]:
#loop through file

columns = [] #initiate columns list
values = [] #initiate values list

for line in file.read().split('\n'): #start for loop for each row in the .txt file
    if transaction_re.search(line): #if row starts with the value(s) mentioned in the regex, keep row
        item = line.split() #split row to extract the item (transaction id etc.) and corresponding value. This returns a list ['item', 'value']
        columns.append(item[0].replace(":","")) #append item to columns, remove ":" for tidiness
        try: #start try except for the values, as some values are empty, which stops Python
            values.append(item[1]) #append value to values
        except IndexError: #indexerrors occur when an index is appointed which is not available. Thus in case no value is entered. 
            values.append("No value entered") #if there is no value, append "No value entered" 

df = pd.DataFrame([columns, values], index=['item', 'value']).T.explode('value') #code from https://stackoverflow.com/questions/66615474/create-a-pandas-dataframe-from-two-lists-column-1-is-first-list-column-2-is-se
#explode() method is used to transform each element of a list to a separate record.

df.head() #print first five rows of the df to see what we are dealing with


,item,value
0,transaction_date,xDateTimeTz
1,transaction_id,notEmpty
2,transaction_message,notEmpty
3,transaction_currency,No value entered
4,transaction_amount,notEmpty


In [342]:
#transforming the df, so the items are columns and the values are the rows (easier if you want to loop through multiple files instantaneously)

#transpose
df = df.transpose() #columns, rows as explained above

#column names are the first row
df.columns = df.iloc[0] 

#remove the first row, as these are the column names
df = df[1:1]

#transform date to date format
#df["transaction_date"][0] = datetime.datetime.strptime(df["transaction_date"][0],"%d/%m/%Y").date() #code from https://stackoverflow.com/questions/2803852/python-date-string-to-date-object

#drop // columns, which was erroneously extracted
df.drop("//",axis=1,inplace=True)

df.head()


item,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,transaction_direction,transaction_status,instrument_type,originator_full_name,...,beneficiary_first_name,beneficiary_middle_names_patronymic,beneficiary_last_name,beneficiary_address,beneficiary_country,beneficiary_account_number,beneficiary_branch_id,beneficiary_bic,beneficiary_fi_name,beneficiary_fi_country


In [343]:
#set the right datatypes

df['transaction_date'] = pd.to_datetime(df['transaction_date'])

df = df.astype({'transaction_id':'string',
                'transaction_message':'string',
                'transaction_currency':'string',
                'transaction_amount':'float',
                'transaction_type':'string',
                'transaction_direction':'string',
                'transaction_status':'string',
                'instrument_type':'string',
                'originator_full_name':'string',
                'originator_first_name':'string',
                'originator_middle_names_patronymic':'string',
                'originator_last_name':'string',
                'originator_address':'string',
                'originator_country':'string',
                'originator_account_number':'string',
                'originator_branch_id':'string',
                'originator_bic':'string',
                'originator_fi_name':'string',
                'originator_fi_country':'string',
                'outgoing_intermediary_fi_bic':'string',
                'beneficiary_full_name':'string',
                'beneficiary_first_name':'string',
                'beneficiary_middle_names_patronymic':'string',
                'beneficiary_branch_id':'string',
                'beneficiary_bic':'string',
                'beneficiary_fi_name':'string',
                'beneficiary_fi_country':'string'})

#df.dtypes


In [344]:
#close the file 
file.close()

Fill the dataframe with  SWIFT MT103 messages.

In [345]:
MESSAGE1 = (
    "{1:F01ASDFJK20AXXX0987654321}"
    "{2:I103ASDFJK22XXXXN}"
    "{4: :20:20180101-ABCDEF :23B:GHIJ :32A:180117CAD5432,1 :33B:EUR9999,0 :50K:/123456-75901 SOMEWHERE New York 999999 GR :53B:/20100213012345 :57C://SC200123 :59:/201001020 First Name Last Name a12345bc6d789ef01a23 Nowhere NL :70:test reference test reason payment group: 1234567-ABCDEF :71A:SHA :77B:Test this -}"  # NOQA: E501
)
MESSAGE2 = (
    "{1:F01QWERTY22AXXX1234567890}"
    "{2:I103QWERTY33XXXXA7}"
    "{3:{108:MT103}}"
    "{4:\n:20:1234567-8901\n:23B:ABCD\n:32A:000625EUR1000,00\n:33B:EUR1000,00\n:50K:COMPANY NAME\nNAPLES\n:52A:ABCDEFGH123\n:53A:ABCDEF12\n:54A:ABCDEF1G\n:57A:ABCDEFGHIJK\n:59:/20061120050500001A01234\nBENEFICIARY NAME\n:70:/REMITTANCE INFO\n:71A:SHA\n-}"  # NOQA: E501
)
MESSAGE3 = (
    "{1:F01QWERTY22AXXX1234567890}"
    "{2:I103QWERTY33XXXXA7}"
    "{3:{113:SEPA}{111:001}{121:d2d62e74-4f7d-45dc-a230-85fa259e1694}}"
    "{4: :20:123456-ABCDEF001 :23B:GHIJ :32A:123456GBP10000,00 :33B:GBP10000,00 :50K:/This is arbitrary text :52D:/123456-78900 More arbitrary text :53B:/12345678901234 :57C://AB123456 :59:/12345678 Even more arbitrary text :70:abc - 12.34 more txt 20190115-ABCDEF :71A:SHA :72:/INS/ABCDEF01 -}"  # NOQA: E501
)
MESSAGE4 = (
    "{1:F01AAAAGRA0AXXX0057000289}"
    "{2:O1030919010321BBBBGRA0AXXX00570001710103210920N}"
    "{3:{108:MT103 003 OF 045}{121:c8b66b47-2bd9-48fe-be90-93c2096f27d2}}"
    "{4:\n:20:5387354\n:23B:CRED\n:23E:PHOB/20.527.19.60\n:32A:000526USD1101,50\n:33B:USD1121,50\n:50K:FRANZ HOLZAPFEL GMBH\nVIENNA\n:52A:BKAUATWW\n:59:723491524\nC. KLEIN\nBLOEMENGRACHT 15\nAMSTERDAM\n:71A:SHA\n:71F:USD10,\n:71F:USD10,\n:72:/INS/CHASUS33\n-}"
    "{5:{MAC:75D138E4}{CHK:DE1B0D71FA96}}"
)

In [346]:
#set directory with files to loop through

In [353]:
#initiate lookup variables
swift_re = re.compile(r':[A-Za-z0-9?]+:*') #regex to identify the items which are needed in the dataframe



swift_re.findall(MESSAGE4)


[':F01AAAAGRA0AXXX0057000289',
 ':O1030919010321BBBBGRA0AXXX00570001710103210920N',
 ':MT103',
 ':c8b66b47',
 ':20:',
 ':23B:',
 ':23E:',
 ':32A:',
 ':33B:',
 ':50K:',
 ':52A:',
 ':59:',
 ':71A:',
 ':71F:',
 ':71F:',
 ':72:',
 ':75D138E4',
 ':DE1B0D71FA96']

In [355]:
#loop through files and look for re compiled information, append this info to df
for line in MESSAGE4.split('\n'): #start for loop for each row in the .txt file
    print(line)



{1:F01AAAAGRA0AXXX0057000289}{2:O1030919010321BBBBGRA0AXXX00570001710103210920N}{3:{108:MT103 003 OF 045}{121:c8b66b47-2bd9-48fe-be90-93c2096f27d2}}{4:
:20:5387354
:23B:CRED
:23E:PHOB/20.527.19.60
:32A:000526USD1101,50
:33B:USD1121,50
:50K:FRANZ HOLZAPFEL GMBH
VIENNA
:52A:BKAUATWW
:59:723491524
C. KLEIN
BLOEMENGRACHT 15
AMSTERDAM
:71A:SHA
:71F:USD10,
:71F:USD10,
:72:/INS/CHASUS33
-}{5:{MAC:75D138E4}{CHK:DE1B0D71FA96}}


Use this structure to either write python functions or sql code to identify the following risk 
patterns (75 points)

> a. Round Amount payments - 15 points

In [349]:
amount = int(df["transaction_amount"][0])

if amount % 1000 == 0:
    df["isRound"] = True
else:
    df["isRound"] = False


IndexError: index 0 is out of bounds for axis 0 with size 0

> b. Payments from high risk countries -15 points (https://www.fatfgafi.org/en/countries/black-and-grey-lists.html) to tax havens 
(https://fsi.taxjustice.no/fsi/2022/world/score/top)

In [ ]:
originator_country = df["originator_country"][0]
beneficiary_country = df["beneficiary_country"][0]

high_risk_countries = ["NC", "IR", "MY", "AL", "BA", "BU", "CA", "CO", "GI", "HA", "JA", "JO", "MA", "MO", "NI", "PA", "PH", "SE", "SA", "SS", "SY", "TA", "TU", "UG", "UA", "YE"]
tax_havens = ["VI", "AN", "BO", "UA", "AL", "PU", "CU"] 

if originator_country in high_risk_countries and beneficiary_country in tax_havens:
    df["suspCountries"] = True
else:
    df["suspCountries"] = False

print(df["suspCountries"][0])

True


> Smurfing -10 points

In [ ]:
#som per originator_account_number
#if over past X days at least N cash deposits were made with an amount between the fix threshold (10k) and [100 - CD(n) * 2%] of that amount (38:29)
import datetime

date_thres = 7 #X
cd_thres = 2 #N
fixed_threshold = 10000
var_threshold = fixed_threshold - cd_thres * 0.02 #create dynamic threshold within 2% of the original threshold times the number of transactions
reference_date = datetime.date(2023,3,20)
transaction_date = df["transaction_date"][0]

#we will work based on three IF's: data threshold (number of days between transactions date and reference date), amount threshold (if between two thresholds 'True') and cash deposits (check the number of transactions versus set threshold of transactions) 
#first we will check the IF's on transaction level, which are the days between transactions and amount.

#IF check if transaction date is within 7 days of reference date
datetime.timedelta(7) #initiate timedelta with 7 
if -date_thres <= (transaction_date-reference_date).days: #calculate the days between the ref. date and trans. date, if smaller or equal to the data threshold continue
    df["inDataThres"] = True
else:
    df["inDataThres"] = False

print("The transaction is within the date threshold: " + str(df["inDataThres"][0]))

#IF check threshold
if fixed_threshold >= amount and var_threshold <= amount:
    df["inAmtThres"] = True
else:
   df["inAmtThres"] = False

print("The transaction is within the amount threshold: " + str(df["inAmtThres"][0]))

#now we will check whether the total count of transactions is within the threshold. For this we group the DF.
#IF check the number of transactions (returns one as we only have 1 example in the database, consequently assumes multiple MT103 transactions in the database)
count = df.groupby(['originator_account_number'])['transaction_amount'].count()[0] #counts the number of transactions, grouped by originator account number If it exceeds the set threshold, 'True'.

if count > cd_thres:
    df["over_cd_thres"] = True
else:
    df["over_cd_thres"] = False
    
print("The transaction is within the count threshold: " + str(df["over_cd_thres"][0]))


The transaction is within the date threshold: True
The transaction is within the amount threshold: False
The transaction is within the count threshold: False


> Nesting -10 points

In [ ]:
#for nesting we will assess the number of origin accounts that transfer money to the beneficiary. 

count_nesting = df.groupby(['beneficiary_account_number'])['originator_account_number'].count()[0] #counts the number of transactions that transfer money to the beneficiary, grouped by beneficiary account number.
print("The number of accounts transferring money to the beneficiary is: " + str(count_nesting))

count_nesting_un = df.groupby(['beneficiary_account_number'])['originator_account_number'].nunique()[0] #counts the number of unique account numbers that transfer money to the beneficiary, grouped by  beneficiary account number.
print("The number of unique accounts transferring money to the beneficiary is: " + str(count_nesting_un))



The number of accounts transferring money to the beneficiary is: 1
The number of unique accounts transferring money to the beneficiary is: 1


> Non-adherence to FATF Recommendation 16 -10 points

In [ ]:
#FATF basically states that all info should be there in a MT103 file. In case the value notEmpty (which is a mandatory field) is present, NonComplFATF is 'True'

emptycols = []

for item in df:
    if df[item][0] == "notEmpty": #check for notEmpty
        df["NonComplFATF"] = True
        emptycols.append(item)
    elif df[item][0] == "": #check for just empty items
        df["NonComplFATF"] = True
        emptycols.append(item)
    else:
        df["NonComplFATF"] = False 

if (df["NonComplFATF"][0]) == False:
    print("The transaction does comply with FATF")
else:
    print("The transaction does not comply with FATF")

print("------------------------------------------------------")


print("The following columns miss values: ")
for emptycol in emptycols: print(emptycol)


The transaction does comply with FATF
------------------------------------------------------
The following columns miss values: 
transaction_message
transaction_type
originator_first_name
originator_middle_names_patronymic
originator_last_name
originator_bic
originator_fi_name
originator_fi_country
beneficiary_full_name
beneficiary_first_name
beneficiary_middle_names_patronymic
beneficiary_last_name
beneficiary_address
beneficiary_account_number
beneficiary_bic
beneficiary_fi_name
beneficiary_fi_country


> f. (https://www.fatfgafi.org/content/dam/recommandations/pdf/FATF%20Recommendations%20201
2.pdf.coredownload.inline.pdf) - 5 points

> g. Shell company characteristics using address and name data. - 5 points

In [ ]:
#to identify a shell company we assess the address and name data. In case it is missing (or empty or notEmpty), we will identify the transaction as isShellComp = True. We will assess the fields originator_full_name and originator_address, as these are mandatory (notEmpty) fields.
 
if (df["originator_address"][0]) == "notEmpty" or (df["originator_address"][0]) == "" or (df["originator_full_name"][0]) == "notEmpty" or (df["originator_full_name"][0]) == "":
    df["isShellComp"] = True
else:
    df["isShellComp"] = False

if df["isShellComp"][0] == True:
    print("The transaction could originate from a Shell company")
else:
    print("The transaction does not originate from a Shell company")



The transaction does not originate from a Shell company


> Trade based money laundering - 5 points
(https://www.fatf-gafi.org/content/fatfgafi/en/publications/Methodsandtrends/Trade-based-money-launderingindicators.html, https://stats.wto.org/dashboard/tradeconnectivity_en.html)

In [ ]:
df.to_excel('MT103_analysis.xlsx')